# Hierarchical ($\mathcal{H}$-)Matrices

In this section we will discuss hierarchical matrices (or $\mathcal{H}$-matrices). These allow the representation of and matrix vector products with the matrix of Green's functions interactions with a complexity of $\mathcal{O}(N\log N)$, where $N$ is the number of evaluation points.

Another nice feature of $\mathcal{H}$-matrices is that it is straight forward to design algorithms for an approximate $LU$ decomposition and approximate inverse of these matrices with also near-linear complexity.

The disadvantage of $\mathcal{H}$-matrices is that compared to Fast Multipole or $\mathcal{H}^2$ matrix techniques they do not have linear complexity (the $\log$ factor becomes important for very large problems), and they tend to have also in practice larger memory requirements than these techniques.

Nevertheless, the algorithmic implementation is very simple and parallelisation is easy. This is why they are still very popular. Moreover, in many practical applications it seems the case that also for oscillatory problems they keep good performance into the medium wavenumber regime (though their asymptotic complexity is not optimal here).

In the literature $\mathcal{H}$-matrix technqiues are also often called ACA methods. The historical reason is that $\mathcal{H}$-matrices are often implemented using adaptive cross approximation. Nevertheless, this is an inaccurate description as ACA is merely the algorithm that low-rank compresses the blocks in an $\mathcal{H}$-matrix. Other low-rank compression techniques can equally be applied.

## From block-cluster trees to $\mathcal{H}$-matrices.



$\mathcal{H}$-matrices are algebraic structures operating on block-cluster trees. Assume we have target and source index sets $I$ and $J$, associated cluster trees $T_I$ and $T_J$, and block-cluster tree $T_{I\times J}$.

Let $G$ be the matrix of Green's function interactions $G[i, j] = g(x_i, y_j)$. The compressed $\mathcal{H}$-matrix representation of $G$ is defined over the block-cluster tree $T_{I\times J}$ in the following way.

- If the node $\tau\times \nu$ is not admissible then $\mathcal{H}_{\tau\times\nu}(G) := G[I_\tau, J_\nu]$, that is the $\mathcal{H}$-matrix stores the direct dense representation of the interactions between $I_\tau$ and $J_\nu$.
- If the node $\tau\times \nu$ is admissible then $\mathcal{H}_{\tau\times\nu}(G):= AB^H$, where $G[I_\tau, J_{\nu}]\approx AB^H$ is a low-rank representation of rank at most $k$.

Hence, in the below representation of a block-cluster tree every green box has a low-rank representation, and every red box has a dense representation.

![hmatrix](./block_cluster_tree.png)